<h1 align=center><font size="5">Comparar modelos de classificação</font></h1>


Neste notebook, você usará os 4 classificadores vistos até aqui: Regressão Logística, SVM, KNN e Árvore de decisão para comparar seus resultados em um conjuntos de dados sintéticos, criados apenas para entendimento e exercício.

Através da biblioteca <code>scikit-learn</code> você verá que é bem fácil fazer o treinamento e adaptar o código para receber diferentes modelos de forma compacta sem precisar alterar muito o código.

<h2>Importação das bibliotecas</h2>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


<h2 id="load_dataset">Criação dos dados sintéticos</h2>

Os dados para esse notebook serão pontos divididos em duas classes. Serão criados 3 conjuntos aleatórios diferentes, com o auxílio das funções <code>make_moons</code>, <code>make_circles</code> e <code>make_classification</code> da <code>scikit-learn</code>. Essas funções são preprogramadas para gerar dados em formatos específicos que podem ser usados para testes de modelos simples.

Cada dataset é formado pela tupla (X, y), onde X são os pontos de entrada e y os de saída.  

In [ ]:

names = ["KNN", "SVM", "Árvode de decisão", "Regressão Logística"]

classifiers = [
    KNeighborsClassifier(3), 
    SVC(kernel="linear", C=0.025), 
    DecisionTreeClassifier(max_depth=5), 
    LogisticRegression(C=0.01, solver='liblinear')]

datasets = [make_classification(n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1),
            make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1)
            ]

O processo se dará da seguinte forma:

- Será feito um laço para passar por cada um dos 3 dataset criados, nessa etapa os dados serão separados entre conjunto de dados e conjunto de treino.
- Dentro do primeiro laço haverá um segundo que irá passar pelos modelos de classficação, aqui serão feitos os treinamentos.

Com isso, cada modelo passará por cada dataset em sequência.

A visualização será dada pela função <code>scatter</code>, as classes serão identificadas por cores.

A final, será possível ver também os limites que cada modelo gerou para cada dataset.

In [ ]:
figure = plt.figure(figsize=(27, 9))
i = 1
# Laço em cima dos datasets
for ds_cnt, ds in enumerate(datasets):
    # Separação dos datasets em treino/teste
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

    # Plotando os datasets
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Dados de entrada")
    # Plota os dados de teste
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
    # Plota os dados de treinamento
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # Laço em cima dos modelos
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # [Curiosidade] Essa parte está fazendo a superfície de decisão, onde 
        # é possível ver como cada modelo está fazendo a classificação dos dados.
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plota os dados de treino
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
        # Plota os dados de teste
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'), size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.show()